In [46]:
import numpy as np

from torch.utils.data import DataLoader, TensorDataset
import torch
import torch.nn as nn
from torch.nn import functional as F
from pytorch_lightning.trainer.trainer import Trainer
from sklearn.model_selection import train_test_split
from pytorch_lightning.core import LightningModule
from pytorch_lightning.callbacks import EarlyStopping

In [16]:
def load_coffee_data():
    """ Creates a coffee roasting data set.
        roasting duration: 12-15 minutes is best
        temperature range: 175-260C is best
    """
    rng = np.random.default_rng(2)
    X = rng.random(400).reshape(-1,2)
    X[:,1] = X[:,1] * 4 + 11.5          # 12-15 min is best
    X[:,0] = X[:,0] * (285-150) + 150  # 350-500 F (175-260 C) is best
    Y = np.zeros(len(X))
    
    i=0
    for t,d in X:
        y = -3/(260-175)*t + 21
        if (t > 175 and t < 260 and d > 12 and d < 15 and d<=y ):
            Y[i] = 1
        else:
            Y[i] = 0
        i += 1

    return (X, Y.reshape(-1,1))

In [57]:
X,Y = load_coffee_data()
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)


class MyNeuralNetwork(LightningModule):
    def __init__(self):
        super(MyNeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.model = nn.Sequential(
            nn.Linear(2, 3),
            nn.ReLU(),
            nn.Linear(3, 1),
            nn.Sigmoid(),
        )

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self.forward(x)
        loss = F.binary_cross_entropy(y_pred, y)
        self.log('train_loss', loss)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self.forward(x)
        loss = F.binary_cross_entropy(y_pred, y)
        self.log('val_loss', loss)
        return loss

    def forward(self, x):
        x = self.flatten(x)
        return self.model(x)
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

In [58]:
features = torch.tensor([X_test[0]], dtype=torch.float32)
model = MyNeuralNetwork()
output = model(features)
print(output)

tensor([[1.]], grad_fn=<SigmoidBackward0>)


In [59]:
early_stop_callback = EarlyStopping(
   monitor='val_loss',
   min_delta=0.01,
   patience=3,
   verbose=False,
   mode='min'
)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
Y_train_tensor = torch.tensor(Y_train, dtype=torch.float32)

train_dataset = TensorDataset(X_train_tensor, Y_train_tensor)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
Y_val_tensor = torch.tensor(Y_val, dtype=torch.float32)

val_dataset = TensorDataset(X_val_tensor, Y_val_tensor)
val_dataloader = DataLoader(val_dataset, batch_size=32)

trainer = Trainer(callbacks=[early_stop_callback], max_epochs=100, log_every_n_steps=1)
trainer.fit(model, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)

X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

predictions = model(X_test_tensor)

# Print the predictions
print(predictions[:3])


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name    | Type       | Params
---------------------------------------
0 | flatten | Flatten    | 0     
1 | model   | Sequential | 13    
---------------------------------------
13        Trainable params
0         Non-trainable params
13        Total params
0.000     Total estimated model params size (MB)


Epoch 12: 100%|██████████| 4/4 [00:00<00:00, 114.28it/s, v_num=19]          
tensor([[0.0089],
        [0.0223],
        [0.0297]], grad_fn=<SliceBackward0>)
